# Model

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Data Preprocessing

In [9]:
from sklearn.preprocessing import OneHotEncoder 

In [3]:
recs = pd.read_csv('data/pruned.csv')

In [4]:
recs.head()

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
0,534380,0,0,2022-10-08,True,40.6,10531,22
1,42700,6,2,2019-10-19,False,5.9,185293,27
2,602960,0,0,2022-01-05,True,41.9,319249,58
3,976730,0,0,2021-11-25,False,21.1,747545,67
4,1091500,2,0,2022-10-30,True,18.1,2113544,127


In [7]:
USED_COLS = ['app_id', 'is_recommended', 'user_id']

recs = recs[USED_COLS]
recs.head()

,app_id,is_recommended,user_id
0,534380,True,10531
1,42700,False,185293
2,602960,True,319249
3,976730,False,747545
4,1091500,True,2113544


In [11]:
M = recs.pivot(columns='app_id', index='user_id', values='is_recommended')

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
M

## Simple Model

In [ ]:
class AveragePredictor:
    def fit():
        pass